## 一周内周KDJ金叉，期间日KDJ金叉且无死叉

In [ ]:
import akshare as ak
import pandas as pd
from datetime import datetime, timedelta
import time
import matplotlib.pyplot as plt
import shutil
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
import os

current_date = datetime.now() + timedelta(days=1)
one_week_ago = current_date - timedelta(days=7)  # 计算一周之前的日期

# 计算KDJ指标
def calculate_kdj(df, n=9):
    """计算 KDJ 指标"""
    low_min = df["low"].rolling(window=n).min()
    high_max = df["high"].rolling(window=n).max()
    rsv = 100 * (df["close"] - low_min) / (high_max - low_min)
    df["K"] = rsv.ewm(com=2).mean()  # K线
    df["D"] = df["K"].ewm(com=2).mean()  # D线
    df["J"] = 3 * df["K"] - 2 * df["D"]  # J线
    return df

# 计算EMA
def calculate_ema(data, span):
    """计算EMA"""
    return data.ewm(span=span, adjust=False).mean()

# 计算MACD
def calculate_macd(df, short=12, long=26, signal=9):
    """计算MACD指标"""
    df["EMA_short"] = calculate_ema(df["close"], short)
    df["EMA_long"] = calculate_ema(df["close"], long)
    df["DIF"] = df["EMA_short"] - df["EMA_long"]
    df["DEA"] = calculate_ema(df["DIF"], signal)
    df["MACD_hist"] = 2 * (df["DIF"] - df["DEA"])
    df["Golden_Cross"] = (df["DIF"].shift(1) <= df["DEA"].shift(1)) & (
        df["DIF"] > df["DEA"]
    )
    return df

# 按周重新采样，重新计算周K线数据的KDJ
def calculate_weekly_kdj(df):
    """计算周K线的KDJ指标"""
    df["date"] = pd.to_datetime(df["date"])  # 确保 date 列是 datetime 格式
    df = df.set_index("date")  # 把 date 设为索引
    
    weekly_data = (
        df.resample("W-FRI")  # 取当周五的数据
        .agg({"open": "first", "high": "max", "low": "min", "close": "last"})
        .dropna()
    )

    # 获取当前日期
    today = pd.Timestamp.today().normalize()  # 只取日期部分，去掉时间
    last_weekly_index = weekly_data.index[-1]  # 取最后一个周五索引

    # 如果今天不是周五，修改最后一行索引为今天
    if today != last_weekly_index:
        weekly_data = weekly_data.rename(index={last_weekly_index: today})

    return calculate_kdj(weekly_data)


def find_kdj_golden_cross(df):
    """查找 KDJ 金叉和死叉，并判断当日股票涨跌"""
    df["Golden_Cross"] = (df["K"] > df["D"]) & (df["K"].shift(1) <= df["D"].shift(1))
    df["Dead_Cross"] = (df["K"] < df["D"]) & (df["K"].shift(1) >= df["D"].shift(1))

    # 判断股票涨跌：今日收盘价高于昨日收盘价则为上涨（Up），否则为下跌（Down）
    df["Price_Change"] = df["close"].diff()  # 计算收盘价变化
    df["Trend"] = df["Price_Change"].apply(lambda x: "Up" if x > 0 else "Down")

    return df


# 查找最近金叉时段 且 今日股票是涨的
def get_golden_to_dead_cross_periods(df):
    """从金叉开始到死叉结束提取日期段"""
    periods = []
    start_date = None  # 用于记录金叉的开始日期

    for i in range(len(df)):
        if df["Golden_Cross"].iloc[i]:  # 如果是金叉
            if start_date is None:  # 开始记录金叉日期
                start_date = df.index[i]
        elif df["Dead_Cross"].iloc[i]:  # 如果是死叉
            if start_date is not None:  # 如果已记录金叉
                end_date = df.index[i]
                # 不再保存金叉-死叉日期段
                # periods.append((start_date, end_date))  # 保存当前金叉-死叉日期段
                start_date = None  # 重置金叉开始日期

    # 如果循环结束时还有未匹配的金叉（没有死叉）
    if start_date is not None:
        # 检查是否小于7天
        is_within_seven_days = ((df.index[-1] - start_date)).days <= 7

        if is_within_seven_days:
            periods.append((start_date, df.index[-1]))  # 配对到最后一行的日期

    return periods  # 返回金叉日期段


def filter_golden_cross_without_dead_cross(data, periods):
    # # 确保索引是 DatetimeIndex
    # if not isinstance(data.index, pd.DatetimeIndex):
    #     data = data.copy()
    #     data["date"] = pd.to_datetime(data["date"])  # 确保日期格式正确
    #     data = data.set_index("date")
    
    # 提取金叉日期
    golden_cross_dates = data[data["Golden_Cross"]]

    # 检查周 KDJ 的金叉段内，无死叉条件
    filtered_dates = []

    # 查找最近日线kdj金叉日期
    last_golden_cross_date = golden_cross_dates.iloc[-1]["date"]

    # 判断和今日时间差
    is_recent = (current_date - last_golden_cross_date).days < 2

    if periods and is_recent:    
        filtered_dates.append(last_golden_cross_date)

    # 判断今日股票涨跌的
    if data["Trend"].iloc[-1] == "Down":
        return []

    return filtered_dates


# 查找符合的股票
def get_recent_golden_cross_dates(stock_code):
    data = download_with_retry(stock_code)

    if data.empty:  # 检查是否下载失败
        print(f"Failed to download data for {stock_code}. Skipping...")
        return []

    # 换成ak了，不需要去掉第二层的ticker
    # data.columns = data.columns.droplevel(1)  # 去掉第二层的 ticker，变为单层索引

    data = calculate_kdj(data)  # 计算日 KDJ
    weekly_data = calculate_weekly_kdj(data)  # 计算周 KDJ
    weekly_data = find_kdj_golden_cross(weekly_data)  # 查找周 K线金叉
    data = find_kdj_golden_cross(data)  # 查找日线 KDJ 金叉和死叉
    periods = get_golden_to_dead_cross_periods(
        weekly_data
    )  # 提取周 KDJ 金叉到死叉的日期段

    # 获取符合条件的金叉日期
    filtered_golden_cross_dates = filter_golden_cross_without_dead_cross(data, periods)

    # 将 filtered_golden_cross_dates 转换为 pandas 的 DatetimeIndex 以便筛选
    filtered_golden_cross_dates = pd.to_datetime(filtered_golden_cross_dates)

    # 筛选出最近一周内的日期
    recent_dates = [
        date.strftime("%Y-%m-%d")
        for date in filtered_golden_cross_dates
        if one_week_ago <= date <= current_date
    ]

    return recent_dates


# 生成股票代码 名称 词典
def generate_stock_dict(filename):
    stock_dict = {}

    with open(filename, "r", encoding="utf-8") as file:
        # 读取文件内容
        content = file.read()

        # 按照股票名和股票代码的格式分割
        stock_info = content.split(")")  # 按照')'分割

        for info in stock_info:
            if "(" in info:  # 如果包含'('，说明这是一个有效的股票信息
                code_name = info.split("(")  # 通过'('分割代码和名称
                if len(code_name) == 2:
                    code = code_name[1]
                    name = code_name[0].strip()
                    stock_dict[code] = name

    return stock_dict

# 带有重试机制的 yfinance 数据下载函数。
def download_with_retry(stock_code, max_retries=2, retry_delay=3):
    end_date = current_date
    for attempt in range(max_retries):
        try:
            df = ak.stock_zh_a_cdr_daily(
                symbol=stock_code,
                start_date="2024-01-01",
                end_date=end_date.strftime("%Y-%m-%d")
            )
            if not df.empty:  # 检查数据是否下载成功
                return df
            else:
                raise ValueError("Empty DataFrame returned.")
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {stock_code}: {e}")
            if attempt < max_retries - 1:
                time.sleep(retry_delay)  # 等待后重试
                end_date = end_date - timedelta(1)
            else:
                print(f"Max retries reached for {stock_code}. Skipping...")
    return pd.DataFrame()  # 如果失败，返回空 DataFrame

# 输出最终excel文件
def output_excel(all_golden_cross_dates, output_file):
    filename = "stock_name.txt"  # 替换新文件路径
    stock_dict = generate_stock_dict(filename)
    num = 0
    data = []
    # 遍历所有股票金叉数据
    for stock_code, dates in all_golden_cross_dates.items():
        if dates:  # 如果有金叉日期
            num += 1
            cur_stock_code = str(stock_code[-6:])  # 去掉市场后缀，保留6位股票代码
            stock_name = stock_dict.get(cur_stock_code, "未知")  # 获取股票名称
            data.append(
                {
                    "股票代码": cur_stock_code,
                    "股票名称": stock_name,
                    "一周内金叉日期": dates[0],  # 最近一周金叉日期
                }
            )

    df = pd.DataFrame(data)
    summary_row = pd.DataFrame(
        {
            "股票代码": [f"总计金叉股票个数：{num}"],
            "股票名称": [""],
            "一周内金叉日期": [""],
        }
    )  # 添加总计行到 DataFrame 的顶部

    df = pd.concat([summary_row, df], ignore_index=True)

    # output_file = "E:\GRADUATE\project\stock_info\golden_cross_summary1.xlsx"
    df.to_excel(output_file, index=False, sheet_name="金叉统计")
    print(f"Excel 文件已生成: {output_file}")


def process_stock(stock_code):
    time.sleep(random.uniform(0.1, 0.2))  # 随机延时
    try:
        recent_golden_cross_dates = get_recent_golden_cross_dates(stock_code)
        return stock_code, recent_golden_cross_dates
    except Exception as e:
        print(f"Error processing {stock_code}: {e}")
        return stock_code, []


# 替换原来的循环处理部分
def process_stocks(stock_list):
    all_golden_cross_dates = {}
    # 创建线程池，max_workers 可以根据需要调整
    with ThreadPoolExecutor(max_workers=10) as executor:
        # 提交所有任务
        future_to_stock = {
            executor.submit(process_stock, stock): stock for stock in stock_list
        }

        # 获取结果
        for future in as_completed(future_to_stock):
            stock_code, dates = future.result()
            all_golden_cross_dates[stock_code] = dates

    return all_golden_cross_dates

In [ ]:
file_name = "stock_code.txt"
with open(file_name, "r") as file:
    stock_list = [line.strip() for line in file if line.strip()]

# # 限制股票数量
# stock_list = stock_list[:100]

all_golden_cross_dates = process_stocks(stock_list)

In [69]:
filename = "stock_name.txt"  # 替换新文件路径
stock_dict = generate_stock_dict(filename)

# 输出所有股票最近一周的金叉日期
num = 0
for stock_code, dates in all_golden_cross_dates.items():
    if dates:
        num += 1
        cur_stock_code = str(stock_code[-6:])  # 取出六位股票代码
        try:
            stock_name = stock_dict[cur_stock_code]  # 得股票具体名称
        except Exception as e:
            stock_name = "未知"
        print(
            f"股票代码: {cur_stock_code}, 股票名称: {stock_name} 一周内金叉日期: {dates[0]}"
        )

print(f"总计金叉股票个数：{num}")

股票代码: 600063, 股票名称: 皖维高新 一周内金叉日期: 2025-02-24
股票代码: 600080, 股票名称: 金花股份 一周内金叉日期: 2025-02-24
总计金叉股票个数：2


In [ ]:
try:
    folder = Path("output")
    folder.mkdir()
except Exception as e:
    pass

folder_path = "./output"  # 指定文件夹路径
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path):
            os.remove(file_path)
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")

In [32]:
output_file = ".\output\golden_cross_summary.xlsx"
output_excel(all_golden_cross_dates, output_file)

Excel 文件已生成: .\output\golden_cross_summary.xlsx


In [33]:
# 读取 Excel
file_path = ".\output\golden_cross_summary.xlsx"
df = pd.read_excel(file_path, usecols=[0])  # 只读取第一列

# 解决中文乱码
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

# 设置每张图最多显示的行数
rows_per_image = 50

# 按每 50 行划分数据并生成图片
for i in range(0, len(df), rows_per_image):
    subset = df.iloc[i : i + rows_per_image]  # 取出 50 行数据
    fig, ax = plt.subplots(figsize=(5, len(subset) * 0.4))  # 调整图片大小
    ax.axis("tight")
    ax.axis("off")
    ax.table(
        cellText=subset.values, colLabels=subset.columns, cellLoc="center", loc="center"
    )

    # 生成文件名，如 output_0_0.png, output_0_1.png
    output_path = f".\output\output_0_{i // rows_per_image}.png"
    plt.savefig(output_path, bbox_inches="tight", dpi=300)

    # 关闭图像，释放内存
    plt.close(fig)

print("所有图片已生成！")

所有图片已生成！


In [65]:
stock_code = "sh600080"
data = download_with_retry(stock_code)
data = calculate_kdj(data)  # 计算日 KDJ
weekly_data = calculate_weekly_kdj(data)  # 计算周 KDJ
weekly_data = find_kdj_golden_cross(weekly_data)  # 查找周 K线金叉
data = find_kdj_golden_cross(data)  # 查找日线 KDJ 金叉和死叉
periods = get_golden_to_dead_cross_periods(
    weekly_data
)  # 提取周 KDJ 金叉到死叉的日期段

In [66]:
golden_cross_dates = data[data["Golden_Cross"]]
last_golden_cross_date = golden_cross_dates.iloc[-1]["date"]
(current_date - last_golden_cross_date).days < 2
(current_date - last_golden_cross_date).days

1

In [53]:
get_recent_golden_cross_dates(stock_code)

['2025-02-24']

In [10]:
df = weekly_data
"""从金叉开始到死叉结束提取日期段"""
periods = []
start_date = None  # 用于记录金叉的开始日期

for i in range(len(df)):
    if df["Golden_Cross"].iloc[i]:  # 如果是金叉
        if start_date is None:  # 开始记录金叉日期
            start_date = df.index[i]
    elif df["Dead_Cross"].iloc[i]:  # 如果是死叉
        if start_date is not None:  # 如果已记录金叉
            end_date = df.index[i]
            # 不再保存金叉-死叉日期段
            # periods.append((start_date, end_date))  # 保存当前金叉-死叉日期段
            start_date = None  # 重置金叉开始日期

# 如果循环结束时还有未匹配的金叉（没有死叉）
if start_date is not None:
    # 检查是否小于7天
    is_within_seven_days = ((df.index[-1] - start_date)).days < 7

    if is_within_seven_days:
        periods.append((start_date, df.index[-1]))  # 配对到最后一行的日期



In [41]:
data.tail(7)

,date,prevclose,open,high,low,close,volume,amount,K,D,J,Golden_Cross,Dead_Cross,Price_Change,Trend
267,2025-02-14,NaN,5.15,5.19,5.06,5.15,2813100.0,14433429.0,65.684542,63.639394,69.774838,False,False,-0.01,Down
268,2025-02-17,NaN,5.15,5.25,5.11,5.20,3153113.0,16356030.0,69.279891,65.519560,76.800553,False,False,0.05,Up
269,2025-02-18,NaN,5.18,5.19,5.04,5.07,3115400.0,15929618.0,58.931692,63.323604,50.147868,False,True,-0.13,Down
270,2025-02-19,NaN,5.07,5.22,4.98,5.22,3468800.0,17656691.0,65.954461,64.200556,69.462271,True,False,0.15,Up
271,2025-02-20,NaN,5.21,5.74,5.17,5.74,8289714.0,46553928.0,77.302974,68.568029,94.772865,False,False,0.52,Up
272,2025-02-21,NaN,5.74,5.74,5.50,5.50,16940401.0,94500304.0,74.342334,70.492797,82.041407,False,False,-0.24,Down
273,2025-02-24,NaN,5.55,5.62,5.46,5.53,8479911.0,46855409.0,73.684363,71.556652,77.939784,False,False,0.03,Up


In [40]:
weekly_data.tail(7)

,open,high,low,close,K,D,J,Golden_Cross,Dead_Cross,Price_Change,Trend
date,,,,,,,,,,,
2025-01-17,4.86,5.30,4.80,5.16,25.656980,35.818296,5.334348,False,False,0.24,Up
2025-01-24,5.17,5.23,4.87,4.96,22.431114,31.355902,4.581538,False,False,-0.20,Down
2025-01-31,5.02,5.15,4.98,5.01,21.139643,27.950482,7.517964,False,False,0.05,Up
2025-02-07,5.02,5.17,4.94,5.10,21.825054,25.908673,13.657816,False,False,0.09,Up
2025-02-14,5.10,5.28,5.06,5.15,23.141101,24.986149,19.451006,False,False,0.05,Up
2025-02-21,5.15,5.74,4.98,5.50,38.843379,29.605226,57.319685,True,False,0.35,Up
2025-02-24,5.55,5.62,5.46,5.53,52.806901,37.339117,83.742468,False,False,0.03,Up
